In [1]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import org.apache.spark.sql._, types._, functions._
import spark.implicits._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/06/28 07:56:16 INFO SparkContext: Running Spark version 2.4.5
21/06/28 07:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/06/28 07:56:17 INFO SparkContext: Submitted application: Queries Optimization
21/06/28 07:56:18 INFO SecurityManager: Changing view acls to: jovyan
21/06/28 07:56:18 INFO SecurityManager: Changing modify acls to: jovyan
21/06/28 07:56:18 INFO SecurityManager: Changing view acls groups to: 
21/06/28 07:56:18 INFO SecurityManager: Changing modify acls groups to: 
21/06/28 07:56:18 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
21/06/28 07:56:18 INFO Utils: Successfully started service 'sparkDriver' on port 3

21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://ab334eb5ebc5:34759/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1624866980112
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://ab334eb5ebc5:34759/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1624866980112
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://ab334eb5ebc5:34759/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1624866980113
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache

21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.9.9/jackson-module-scala_2.12-2.9.9.jar at spark://ab334eb5ebc5:34759/jars/jackson-module-scala_2.12-2.9.9.jar with timestamp 1624866980124
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9.jar at spark://ab334eb5ebc5:34759/jars/jackson-module-paranamer-2.9.9.jar with timestamp 1624866980125
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/squareup/okhttp3/logging-interceptor/3.11.0/logging-interceptor-3.11.0.jar at spark://ab334eb5ebc5:34759/jars/logging-interceptor-3.11.0.jar with timestamp 1624866980125
21/06/28 07:56:20 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.

Spark UI

import $file.$     

import spark._

import common._

import org.apache.spark.sql.functions._

import org.apache.spark.rdd._

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import org.apache.spark.sql._, types._, functions._

import spark.implicits._

In [2]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

# Query 1: Media de infecciones diarias por país.

Esta consulta utiliza un único conjunto de datos con las infecciones diarias reportadas por cada país.

El objetivo de la query es:
    - Agrupar el conjunto de datos por paises
    - Sumar todos los datos diarios obteniendo el valor acumulado
    - Dividir el valor acumulado entre el número de días reportados

De esta forma obtendremos la media de infecciones diarias por país.

## 1. RDD para la consulta de infecciones diarias por país

In [3]:
val infectionData = spark.sparkContext.textFile("../datasets/data.csv")

infectionData: RDD[String] = ../datasets/data.csv MapPartitionsRDD[1] at textFile at cmd2.sc:1

Creo una funcion para trabajar con un RDD de infecciones

In [4]:
def infections(lines : RDD[String]) : RDD[Infection] =
    lines.map(line => {
      val arr = line.split(",")
      Infection(
        day = arr(1).toInt,
        month = arr(2).toInt,
        year = arr(3).toInt,
        nCases = arr(4).toInt,
        nDeaths = arr(5).toInt,
        country = arr(6),
        continent = arr(10)
      )
    })

defined function infections

Calculo la media de infecciones diarias por país trabajando con pair RDD

In [5]:
  def infectionGrowthAverage(infections : RDD[Infection]) : RDD[(String, Int)]= {

    val countriesAndCases : RDD[(String, Iterable[Int])] = 
      infections.map(x => (x.country,x.nCases))
      .groupByKey()
      
    countriesAndCases.mapValues(x => (x.sum / x.size)).sortBy(_._2)
  }

defined function infectionGrowthAverage

Muestro el resultado y el tiempo de ejecución

In [6]:
def infectionRDD = infections(infectionData)

defined function infectionRDD

In [7]:
def query_RDD = infectionGrowthAverage(infectionRDD)

defined function query_RDD

## 2. Con un DataSet

In [8]:
val infectionDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .csv("../datasets/covidworldwide.csv")
    .as[(String,String,String,String,String,String,String,String,String,String,String,String)]

csv at cmd7.sc:5

1 / 1

infectionDS: Dataset[(String, String, String, String, String, String, String, String, String, String, String, String)] = [dateRep: string, day: string ... 10 more fields]

In [9]:
def query_DS = 
    infectionDS.groupBy($"countriesAndTerritories")
    .agg(avg($"cases"))
    .orderBy("avg(cases)")
    .as[(String,Double)]

defined function query_DS

## 3. Hago los mismos calculos con un DataFrame

#### Creo el DataFrame directamente importando los datos

In [10]:
val dfCovid = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "false")
    .csv("../datasets/covidworldwide.csv")

csv at cmd9.sc:6

1 / 1

dfCovid: DataFrame = [dateRep: string, day: string ... 10 more fields]

Utilizo los métodos de la API DF que incluye uno optimizado para calcular la media.

In [11]:
def query_DF = dfCovid
    .groupBy("countriesAndTerritories")
    .agg(mean("cases"))
    .orderBy("avg(cases)")

defined function query_DF

# 4. Visualización de rendimiento

Gráficamente utilizando plotly

In [12]:
val (x, y) = Seq(
    "RDD" -> runWithOutput(query_RDD.collect),
    "DataSet" -> runWithOutput(query_DS.collect),
    "DataFrame" -> runWithOutput(query_DF.collect)
).unzip

Bar(x, y).plot()

map at cmd4.sc:4

2 / 2

sortBy at cmd4.sc:7

2 / 2

sortBy at cmd4.sc:7

...

200 / 200

collect at cmd11.sc:4

200 / 200

Took 5574


x: Seq[String] = List("RDD", "DataSet", "DataFrame")
y: Seq[Int] = List(3384, 9125, 5574)
res11_1: String = "plot-4e60277f-6c58-41e6-96ed-da7eab5108c2"

# 5. Comparativas de rendimiento

Usando la API de spark

In [13]:
val timeRDD = spark.time(query_RDD.collect())

map at cmd4.sc:4

2 / 2

sortBy at cmd4.sc:7

2 / 2

sortBy at cmd4.sc:7

2 / 2

collect at cmd12.sc:1

2 / 2

Time taken: 1641 ms


timeRDD: Array[(String, Int)] = Array(
  ("Greenland", 0),
  ("British_Virgin_Islands", 0),
  ("\"Bonaire", 0),
  ("Fiji", 0),
  ("Vanuatu", 0),
  ("Saint_Vincent_and_the_Grenadines", 0),
  ("Solomon_Islands", 0),
  ("Holy_See", 0),
  ("Dominica", 0),
  ("Wallis_and_Futuna", 0),
  ("Grenada", 0),
  ("Marshall_Islands", 0),
  ("Anguilla", 0),
  ("Timor_Leste", 0),
  ("Saint_Kitts_and_Nevis", 0),
  ("Falkland_Islands_(Malvinas)", 0),
  ("Montserrat", 0),
  ("Northern_Mariana_Islands", 0),
  ("Seychelles", 0),
  ("Antigua_and_Barbuda", 0),
  ("Laos", 0),
  ("Brunei_Darussalam", 0),
  ("New_Caledonia", 0),
  ("Guernsey", 1),
  ("Saint_Lucia", 1),
  ("Cayman_Islands", 1),
  ("Bermuda", 1),
  ("Isle_of_Man", 1),
  ("Barbados", 1),
  ("Cambodia", 1),
  ("Faroe_Islands", 1),
  ("Bhutan", 1),
  ("Monaco", 1),
  ("Mauritius", 1),
  ("United_Republic_of_Tanzania", 1),
  ("Papua_New_Guinea", 2),
  ("Eritrea", 2),
  ("Taiwan", 2),
...

In [14]:
val timeDS = spark.time(query_DS.collect)

collect at cmd13.sc:1

2 / 2

collect at cmd13.sc:1

200 / 200

collect at cmd13.sc:1

200 / 200

collect at cmd13.sc:1

200 / 200

Time taken: 4643 ms


timeDS: Array[(String, Double)] = Array(
  ("Vanuatu", 0.029411764705882353),
  ("Anguilla", 0.03802281368821293),
  ("Montserrat", 0.048507462686567165),
  ("Wallis_and_Futuna", 0.05084745762711865),
  ("Greenland", 0.07037037037037037),
  ("Falkland_Islands_(Malvinas)", 0.07450980392156863),
  ("Marshall_Islands", 0.0851063829787234),
  ("Holy_See", 0.09454545454545454),
  ("Saint_Kitts_and_Nevis", 0.10227272727272728),
  ("Timor_Leste", 0.11567164179104478),
  ("New_Caledonia", 0.13382899628252787),
  ("Laos", 0.15471698113207547),
  ("Fiji", 0.17037037037037037),
  ("Grenada", 0.25842696629213485),
  ("Solomon_Islands", 0.2833333333333333),
  ("British_Virgin_Islands", 0.2889733840304182),
  ("Dominica", 0.3258426966292135),
  ("Saint_Vincent_and_the_Grenadines", 0.3684210526315789),
  ("Northern_Mariana_Islands", 0.4362934362934363),
  ("Brunei_Darussalam", 0.5448028673835126),
  ("Antigua_and_Barbuda", 0.5481481481481482),
  ("Bonaire, Saint Eustatius and Saba", 0.673151750972762

In [15]:
val timeDF = spark.time(query_DF.collect)

collect at cmd14.sc:1

2 / 2

collect at cmd14.sc:1

200 / 200

collect at cmd14.sc:1

200 / 200

collect at cmd14.sc:1

200 / 200

Time taken: 4125 ms


timeDF: Array[Row] = Array(
  [Vanuatu,0.029411764705882353],
  [Anguilla,0.03802281368821293],
  [Montserrat,0.048507462686567165],
  [Wallis_and_Futuna,0.05084745762711865],
  [Greenland,0.07037037037037037],
  [Falkland_Islands_(Malvinas),0.07450980392156863],
  [Marshall_Islands,0.0851063829787234],
  [Holy_See,0.09454545454545454],
  [Saint_Kitts_and_Nevis,0.10227272727272728],
  [Timor_Leste,0.11567164179104478],
  [New_Caledonia,0.13382899628252787],
  [Laos,0.15471698113207547],
  [Fiji,0.17037037037037037],
  [Grenada,0.25842696629213485],
  [Solomon_Islands,0.2833333333333333],
  [British_Virgin_Islands,0.2889733840304182],
  [Dominica,0.3258426966292135],
  [Saint_Vincent_and_the_Grenadines,0.3684210526315789],
  [Northern_Mariana_Islands,0.4362934362934363],
  [Brunei_Darussalam,0.5448028673835126],
  [Antigua_and_Barbuda,0.5481481481481482],
  [Bonaire, Saint Eustatius and Saba,0.6731517509727627],
  [Seychelles,0.68],
  [Saint_Lucia,1.0],
  [Cambodia,1.0527859237536656],


o bien el framework del cern que nos da más información

In [16]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_RDD.collect())

map at cmd4.sc:4

2 / 2

sortBy at cmd4.sc:7

2 / 2

sortBy at cmd4.sc:7

...

4 / 4

take at stagemetrics.scala:207

1 / 1


Scheduling mode = FIFO
Spark Context default degree of parallelism = 6
Aggregated Spark stage metrics:
numStages => 4
numTasks => 8
elapsedTime => 1194 (1 s)
stageDuration => 1176 (1 s)
executorRunTime => 2026 (2 s)
executorCpuTime => 499 (0.5 s)
executorDeserializeTime => 79 (79 ms)
executorDeserializeCpuTime => 31 (31 ms)
resultSerializationTime => 1 (1 ms)
jvmGCTime => 0 (0 ms)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 8 (8 ms)
resultSize => 8340 (8.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 453246
recordsRead => 61900
bytesRead => 4350839 (4.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 124014
shuffleTotalBlocksFetched => 12
shuffleLocalBlocksFetched => 12
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 401372 (391.0 KB)
shuffleLocalBytesRead => 401372 (391.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
shuffleBytesWritten => 203394 (198.0 

res15: Array[(String, Int)] = Array(
  ("Greenland", 0),
  ("British_Virgin_Islands", 0),
  ("\"Bonaire", 0),
  ("Fiji", 0),
  ("Vanuatu", 0),
  ("Saint_Vincent_and_the_Grenadines", 0),
  ("Solomon_Islands", 0),
  ("Holy_See", 0),
  ("Dominica", 0),
  ("Wallis_and_Futuna", 0),
  ("Grenada", 0),
  ("Marshall_Islands", 0),
  ("Anguilla", 0),
  ("Timor_Leste", 0),
  ("Saint_Kitts_and_Nevis", 0),
  ("Falkland_Islands_(Malvinas)", 0),
  ("Montserrat", 0),
  ("Northern_Mariana_Islands", 0),
  ("Seychelles", 0),
  ("Antigua_and_Barbuda", 0),
  ("Laos", 0),
  ("Brunei_Darussalam", 0),
  ("New_Caledonia", 0),
  ("Guernsey", 1),
  ("Saint_Lucia", 1),
  ("Cayman_Islands", 1),
  ("Bermuda", 1),
  ("Isle_of_Man", 1),
  ("Barbados", 1),
  ("Cambodia", 1),
  ("Faroe_Islands", 1),
  ("Bhutan", 1),
  ("Monaco", 1),
  ("Mauritius", 1),
  ("United_Republic_of_Tanzania", 1),
  ("Papua_New_Guinea", 2),
  ("Eritrea", 2),
  ("Taiwan", 2),
...

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_DS.collect)

collect at cmd16.sc:1

2 / 2

collect at cmd16.sc:1 
 (kill)

137 + 7 / 200

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(query_DF.collect)

# 6. Visualización de datos

In [ ]:
val (x,y) = query_DF
    .collect
    .map(r=>(r(0).toString, r(1).toString.toDouble))
    .toList
    .unzip
Bar(x, y).plot()